# Preprocessing
\
The raw data are point clouds generated by MLS (mobile LiDAR). The very first preprocessing step involves processing by the SLAM algorithm (more specifically, via the application provided by GeoSLAM), and unzip files.

Then, the next part consists in classifying ground points, generating a DTM, and extracting vegetation points in a slice parallel to DTM. This step is carried out using the opensource software Computree, and is detailed in the file computree_steps.xsct2 (it is ready to use, only the input files need to be selected and the output folder specified, parameters can also be viewed and modified).

Note: depending on the device you are using, it may be necessary to tile the point cloud before processing with Computree. In this case, make sure you convert to point format 7 and merge the output tile files before moving on. I recommend using CloudCompare for tiling and for converting .laz to .las.

*Output folder: 'computree_outputs'*

# Clustering
\
*Input folder: 'computree_outputs'*

In [ ]:
# Loading packages and modules
import os
import glob
from clustering import ClEngine, Cluster

las_files_path = 'computree_outputs' # location of preprocessed files
clusters_las_path = 'clusters_las' # location of clustered files
clusters_img_path = 'clusters_img'

spheres_file = 'preprocessing/spheres_coordinates.csv' # file containing plot centre coordinates

figsize = (4,4)
dpi = 75

# Listing all .las files to cluster
las_files = glob.glob('computree_outputs/*.las')

# Listing already clustered files
clustered = glob.glob(clusters_las_path + '/*.las')
clustered_names = [os.path.splitext(os.path.basename(file))[0] for file in clustered]

In [ ]:
for file in las_files:
    
    # Checking if the file is not already clustered
    if os.path.splitext(os.path.basename(file))[0]+'_clusters' not in clustered_names:
        
        cl = ClEngine(file)
        
        # Cluster .las file points
        cl.DBSCAN_clustering(eps=0.05, min_samples=100) # distance parameters are in meters
        
        # Filter clusters given on a minimum number of points and minimum length. Note that filter based on
        # a minimum length take a long to process, set the min_dist param to "None" if you don't want to
        # use it.
        pla.filtering(nb_points=500,
                      coord_file=spheres_file,
                      sep=';',
                      dec=',',
                      distance_from_centre=18,
                      delta=0.05,
                      min_dist=None)
        
        # Draw a (very) basic representation of the clusters
        # cl.draw_clusters()
        
        # Save clustering results in new .las files
        cl.save_clusters_las(folder=clusters_las_path)
        
        # Save clustering results in .png files
        cl.save_clusters_img(folder=clusters_img_path, figsize=figsize, dpi=dpi)

# Image classification

## Image creation
\
First, we need to create images from the previously generated points clouds of cylindrical shapes. Graphs are generated from the points by colouring them according to their z coordinate, and are exported in .png format.

*Input folder: 'shapes_raw'*

In [ ]:
# Loading packages and modules
import os
import glob
import image_creation as imgcreate

# Path of folder with folders containing the files from which to create images
path_raw = 'shapes_raw'
img_path = 'shapes_img'

# Image sizing
image_size_inches = (4, 4) # in inches
dpi = 75

In [ ]:
# Browse all files and create images from each file
for folder in glob.glob(path_raw+'/*'):
    
    folder_name = os.path.splitext(os.path.basename(folder))[0]
    dest = img_path + '/' + folder_name
    
    imgcreate.image_generator(data_folder=folder, img_folder=dest, figsize=image_size_inches, dpi=dpi)

*Output folder: 'shapes_img'*

## NNCLR model building
\
The aim is not to spend our time manually classifying images, so we will be using a NNCLR model, adapted to a small labelled training set. The NNCLR model used here is based on the example of https://keras.io/examples/vision/nnclr/ (see link for more details).

*Input folder: 'NNCLR_data', containing images for training*

```
NNCLR_data
│
├── labelled
│   ├── deadwood
│   │   ├── image1.png
│   │   ├── image2.png
│   │   ├── ...
│   │
│   └── other
│       ├── image1.png
│       ├── image2.png
│       ├── ...
│
└── unlabelled
    └── unlabelled
        ├── image1.png
        ├── image2.png
        ├── ...
```


In [ ]:
import os
import tensorflow as tf
import image_classification as imgclf

device = 'CPU:0' # device used to run tensorflow
model_path = 'NNCLR_data' # path with labelled and unlabelled images for training
save_path = model_path + '/finetuning_model'
batch_size = 32
num_epochs = 50 # max number of epochs, the model will stop automatically when val_p_loss has not increased for 5 epochs
image_size = (300, 300) # in pixels

In [ ]:
with tf.device(device):
    
    # Initialise model hyperparameters. For information:
    # input_shape = (image_size[0], image_size[1], 3)
    # AUTOTUNE = tf.data.AUTOTUNE
    # shuffle_buffer = 5000   
    # temperature = 0.1
    # queue_size = 10000
    # contrastive_augmenter = {
    #     "brightness": 0.5,
    #     "name": "contrastive_augmenter",
    #     "scale": (0.2, 1.0)}
    # classification_augmenter = {
    #     "brightness": 0.2,
    #     "name": "classification_augmenter",
    #     "scale": (0.5, 1.0)}
    # width = 128
    model = imgclf.Model(model_path, image_size, batch_size, num_epochs)
    
    # Prepare training and validation datasets
    model.prepare_dataset()
    
    # Pre-train NNCLR
    model.pretraining()
    
    # Evaluate the model
    model.finetuning(save_path=save_path)

## Identification of deadwood images and conversion into .las files

In [ ]:
import os
import glob
import laspy
import numpy as np
import tensorflow as tf
import shape_processing as sp
import image_classification as imgclf

# Needed paths
model_path = 'NNCLR_data'
save_path = model_path + '/finetuning_model'
path_raw = 'shapes_raw'
path_img = 'shapes_img'
path_dw = 'deadwood'
spheres_file = 'preprocessing/spheres_coordinates.csv'

# Same as before
device = 'CPU:0'
batch_size = 32
num_epochs = 50
image_size = (300, 300)

# Get already classified file names
classified_files = glob.glob(path_dw+'/*.las')
classified_names = [os.path.splitext(os.path.basename(file))[0] for file in classified_files]

In [ ]:
# Re-create model
model = imgclf.Model(model_path, image_size, batch_size, num_epochs)
model.load(save_path)

In [ ]:
# Browse folder for each study area
for folder in glob.glob(path_img+'/*'):
    
    folder_name = os.path.splitext(os.path.basename(folder))[0]
    
    #Check if classification is not already done
    if folder_name + '_deadwood' not in classified_names:
        
        print("Classifying " + folder_name + " shapes.")
        
        # Initialise counters...
        total = 0 # of total number of shapes
        dw = 0 # of identified deadwood shapes
        
        # Initialise new las file
        path_out = path_dw+'/'+folder_name+ '_deadwood.las'
        new_las = laspy.create(point_format=7, file_version="1.4")
        new_las.header.scales = np.array([1.e-05, 1.e-05, 1.e-05])
        new_las.write(path_out)
        
        # Browse all shapes of the study area
        for image in glob.glob(folder+'/*.png'):
            
            # Make a prediction with the model: each image classified as deadwood by the model, and
            # classified as "other" but with a score < treshold are kept in the final point cloud
            if model.prediction(image, treshold=0.8):
                
                image_name = os.path.splitext(os.path.basename(image))[0]
                shape_file = path_raw + '/' + folder_name + '/' + image_name + '.txt'
                shape = sp.shape_processing(shape_file)
                
                # Additional filters
                
                # Filtering by inclination to the vertical
                shape.inclination_filter(angle=40)
                
                # Filtering "flying" branches
                shape.flying_filter()
                
                # Filtering shapes outside the inventory plot
                shape.distance_from_centre(plot_name=folder_name, coord_file='preprocessing/spheres_coordinates.csv')
                
                if not shape.is_filtered():
                    
                    dw += 1
                    
                    # Create .las points
                    points = shape.las_points(header=new_las.header, label=dw)

                    # Append .las points to new file
                    with laspy.open(path_out, mode="a") as las_out:
                        las_out.append_points(points)                
            
            total += 1
        
        print(str(dw)+" shapes classified as deadwood out of "+str(total)+".\n")